In [25]:
import pandas as pd
import scipy as sc
import numpy as np
import matplotlib.pylab as pl
import matplotlib.pyplot as plt
import seaborn as sns # You might need to install this (e.g., pip install seaborn)
from scipy.optimize import leastsq
from sklearn.metrics import r2_score
from lmfit import Minimizer, Parameters, report_fit

import warnings
warnings.filterwarnings("ignore")



In [26]:
df = pd.read_csv("../data/LogisticGrowthData.csv")
df.insert(0, "ID", df.Species + "_" + df.Temp.map(str) + "_" + df.Medium + "_" + df.Citation)

In [27]:
def residuals_logistic(params , Time, PopBio):
    N_0, N_max, r = params
    predict = N_0 * N_max * np.exp(r * Time) / (N_max + N_0 * (np.exp(r*Time) - 1))
    return PopBio - predict

def logistic_fit(Time, PopBio):
    init_params = [PopBio[0], PopBio[-1], 0.5]
    param_lsq = leastsq(residuals_logistic, init_params, args=(Time, PopBio))[0]
    N_0, N_max, r = param_lsq
    PopBio_pred = N_0 * N_max * np.exp(r * Time) / (N_max + N_0 * (np.exp(r*Time) - 1))
    return param_lsq, PopBio_pred

def residuals_gompertz(params, t, data):
    '''Model a logistic growth and subtract data'''
    #Get an ordered dictionary of parameter values
    v = params.valuesdict()
    #Logistic model
    model = v['N_0'] + (v['N_max'] - v['N_0']) * \
    np.exp(-np.exp(v['r_max'] * np.exp(1) * (v['t_lag'] - t) / \
                   ((v['N_max'] - v['N_0']) * np.log(10)) + 1))
    #Return residuals
    return model - data

def Gompertz_fit(Time, PopBio):
    #Create object for parameter storing
    params_gompertz = Parameters()
    # add with tuples: (NAME VALUE VARY MIN  MAX  EXPR  BRUTE_STEP)
    params_gompertz.add_many(('N_0', np.log(PopBio)[0] , True, 0, None, None, None),
                             ('N_max', np.log(PopBio)[-1], True, 0, None, None, None),
                             ('r_max', 0.62, True, None, None, None, None),
                             ('t_lag', 5, True, 0, None, None, None))#I see it in the graph

    #Create a Minimizer object
    minner = Minimizer(residuals_gompertz, params_gompertz, fcn_args=(Time, np.log(PopBio)))
    #Perform the minimization
    fit_gompertz = minner.minimize()
    return fit_gompertz
    

def cal_AIC(y_true, y_pred , p):
    residual = y_true - y_pred 
    RSS = sum(residual**2)
    n = len(y_true)
    AIC = n * np.log(RSS/n) + 2*p
    return AIC
    
    
def cal_AICC(y_true, y_pred , p):
    aic = cal_AIC(y_true, y_pred , p)
    n = len(y_true)
    AICC = aic + 2*p*(p + 1)/(n - p - 1)
    return AICC
    
# def save_figure_fitting(Time, PopBio, PopBio_pred, fit_name , unit, fileName ):
#     plt.scatter( Time, PopBio, label = 'Observation')
#     plt.plot(Time, PopBio_pred, label = 'Logistic fit', c = 'r')
#     plt.xlabel("Time(Hours)")
#     plt.ylabel("Population({})".format(unit))
#     plt.legend()
#     plt.title(fit_name)
#     plt.savefig(fileName, dpi = 500)
#     plt.show()

In [28]:
ID_set = sorted(set(df['ID']))
DFs = []

R2_list = []
AIC_list = []
AICC_list = []

logistic_params = []
Gompertz_params = []
OLS_params = []

ID_list = []
unit_list = []


In [29]:
DFs = []
for i, ID in enumerate(ID_set):
    mask = df.ID == ID
    ID_df = df[mask].reset_index(drop = True)
    unit = df['PopBio_units'][0]
    ID_df = ID_df[ID_df['Time'] > 0]
    ID_df = ID_df[ID_df['PopBio'] > 0]
    ID_df = ID_df.sort_values("Time")
    Time = ID_df['Time'].values
    PopBio = ID_df['PopBio'].values
    
    predict_df = pd.DataFrame({'Time': Time, 'PopBio':PopBio })
    predict_df['PopBio_units'] = unit
    predict_df.insert(0, "ID",  ID)
    
    try:
    
        # Gompertz model
        fit_gompertz = Gompertz_fit(Time, PopBio)
        N_0 = fit_gompertz.params['N_0'].value
        N_max = fit_gompertz.params['N_max'].value
        r_max = fit_gompertz.params['r_max'].value
        t_lag = fit_gompertz.params['t_lag'].value

        log_PopBio_pred = N_0 + (N_max - N_0) * \
            np.exp(-np.exp(r_max * np.exp(1) * (t_lag - Time) / \
                           ((N_max - N_0) * np.log(10)) + 1))

        Gompertz_pred = np.exp(log_PopBio_pred)
        Gompertz_r2 = r2_score(PopBio, Gompertz_pred)  
        Gompertz_AIC = cal_AIC(PopBio, Gompertz_pred, p = 4)
        Gompertz_AICC = cal_AICC(PopBio, Gompertz_pred, p = 4)

        # Logistic model
        param_lsq, Logistic_pred = logistic_fit(Time, PopBio)
        Logistic_r2 = r2_score(PopBio, Logistic_pred)   
        Logistic_AIC = cal_AIC(PopBio, Logistic_pred, p = 3)
        Logistic_AICC = cal_AICC(PopBio, Logistic_pred, p = 3)

        # OLS fit
        fit_linear_OLS = np.polyfit(Time, np.log(PopBio), 3) # degree = 3 as this is a cubic 
        OLS_func = np.poly1d(fit_linear_OLS)
        log_PopBio_pred = OLS_func(Time)
        OLS_pred = np.exp(log_PopBio_pred)
        OLS_r2 = r2_score(PopBio, OLS_pred)
        OLS_AIC = cal_AIC(PopBio, OLS_pred, p = 4)
        OLS_AICC = cal_AICC(PopBio, OLS_pred, p = 4)
        
        # summary 
        predict_df['Gompertz_fit'] = Gompertz_pred
        predict_df['Logistic_fit'] = Logistic_pred
        predict_df['OLS_fit'] = OLS_pred
        DFs.append(predict_df)
        unit_list.append(unit)
        ID_list.append(ID)
        R2_list.append([Gompertz_r2,Logistic_r2, OLS_r2 ])
        AIC_list.append([Gompertz_AIC, Logistic_AIC, OLS_AIC])
        AICC_list.append([Gompertz_AICC, Logistic_AICC, OLS_AICC])
        
        Gompertz_params.append([N_0, N_max, r_max, t_lag])
        logistic_params.append(param_lsq)
        OLS_params.append(fit_linear_OLS)

        # fitting figure
        length_ID = len(ID)
        n_split = 6
        row_length = length_ID // n_split
        title = []
        for i in range(n_split - 1):
            title.append(ID[i*row_length :i*row_length+row_length])
        title.append(ID[i*row_length+row_length:])
        title = '\n'.join(title)

        fileName = "../results/"+ ID[:50] + '.png'

#         plt.figure(figsize = (7,4))
#         plt.scatter( Time, PopBio, label = 'Observation')
#         plt.plot(Time, Logistic_pred, label = 'Logistic fit')
#         plt.plot(Time, Gompertz_pred, label = 'Gompertz fit')
#         plt.plot(Time, OLS_pred, label = 'OLS fit')
#         plt.legend()
#         plt.xlabel("Time(Hours)")
#         plt.ylabel("Population({})".format(unit))
#         plt.title(title)
#         plt.savefig(fileName, dpi = 500)
#         plt.show()
    except:
        pass

In [30]:
concat_pred_df = pd.concat(DFs)
concat_pred_df.to_csv('../sandbox/prediction_result.csv', index = None)

In [32]:
logistic_param_df = pd.DataFrame(logistic_params, columns = ['N_0', 'N_max', 'r'])
logistic_param_df.insert(0,'PopBio_units', unit_list)
logistic_param_df.insert(0,'ID', ID_list)
logistic_param_df['R-square'] = np.array(R2_list)[:,1]
logistic_param_df['AIC'] = np.array(AIC_list)[:,1]
logistic_param_df.to_csv("../sandbox/Logistic_fit_params_result.csv", index = None)

In [33]:
Gompertz_param_df = pd.DataFrame(Gompertz_params, columns = ['N_0', 'N_max', 'r_max', 't_lag'])
Gompertz_param_df.insert(0,'PopBio_units', unit_list)
Gompertz_param_df.insert(0,'ID', ID_list)
Gompertz_param_df['R-square'] = np.array(R2_list)[:,0]
Gompertz_param_df['AIC'] = np.array(AIC_list)[:,0]
Gompertz_param_df.to_csv("../sandbox/Gompertz_fit_params_result.csv", index = None)

In [18]:
OLS_param_df = pd.DataFrame(OLS_params, columns = [ 'Time^3', 'Time^2', 'Time', 'constant'])
OLS_param_df.insert(0,'PopBio_units', unit_list)
OLS_param_df.insert(0,'ID', ID_list)
OLS_param_df['R-square'] = np.array(R2_list)[:,-1]
OLS_param_df['AIC'] = np.array(AIC_list)[:,-1]
OLS_param_df.to_csv("../sandbox/OLS_fit_params_result.csv", index = None)

In [34]:
R2_df = pd.DataFrame(R2_list, index = ID_list, columns = ['logistic R2', 'Gompertz R2', 'OLS R2'])
print("R-square descriptive statistics:")
print(R2_df.describe())

R-square descriptive statistics:
         logistic R2   Gompertz R2      OLS R2
count     256.000000  2.560000e+02  256.000000
mean    -4549.259300  4.038190e-01    0.707242
std     37207.308387  8.328781e-01    0.574374
min   -440587.364119 -6.445801e+00   -5.945408
25%        -9.283264 -5.204726e-12    0.644913
50%         0.064611  8.279722e-01    0.876121
75%         0.944282  9.824879e-01    0.965826
max         0.999494  9.999744e-01    0.999282


In [35]:
AIC_df = pd.DataFrame(AIC_list, index = ID_list, columns = ['logistic AIC', 'Gompertz AIC', 'OLS AIC'])
print("AIC descriptive statistics:")
print(AIC_df.describe())

AIC descriptive statistics:
       logistic AIC  Gompertz AIC      OLS AIC
count    256.000000    256.000000   256.000000
mean     120.279546     71.627217    75.832056
std      260.503789    294.032876   291.141843
min     -170.888631   -326.149248  -338.731770
25%        2.506050    -99.800599   -70.544378
50%       65.893479     58.523064    60.224759
75%      118.114798    121.947773   102.923553
max     1994.219261   1986.751487  1999.649196


In [36]:
AICC_df = pd.DataFrame(AICC_list, index = ID_list, columns = ['logistic AICC', 'Gompertz AICC', 'OLS AICC'])
print("AICC descriptive statistics:")
print(AICC_df.describe())

AICC descriptive statistics:
       logistic AICC  Gompertz AICC     OLS AICC
count     256.000000     256.000000   256.000000
mean      129.968684      75.637110    85.521193
std       259.314611     294.001068   290.971154
min      -168.031488    -325.058339  -336.992640
25%         5.348098     -97.687928   -66.947140
50%        80.926621      63.382734    77.914116
75%       129.523331     125.883346   118.469357
max      1994.493233    1986.914752  1999.923168
